# Daily Stopwatch Data Science: KAThai Visualization

Note: this is a part of visualizing Khan Acadmy Thailand videos information. For more information, see https://khanacademythailand.org

## Stage 1: Ask a question

My objective is to create a visualization tool for Khan Academy Thailand videos.

There is no performance measure for the visualization yet.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

**Note:** in order to work properly, you need to set up a terminal with UTF-8 format. See more [here](http://stackoverflow.com/questions/10561923/unicodedecodeerror-ascii-codec-cant-decode-byte-0xef-in-position-1).

In [1]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy

#Reading directly from Google sheets is complicated. Let's do a simple version by going to a local CSV file. 
#For more information about Google Sheets API, see https://developers.google.com/sheets/api/quickstart/python
'''
from StringIO import StringIO              #Properly read goodle sheets got moved to io in python3.

import requests

#read Google sheets data
r = requests.get('https://docs.google.com/spreadsheets/d/1uJSAtyUTQX9pmbI8YlXg_3Z15vKy3Bu4-wvzmoYOep0&output=csv')
data = r.content

df = pd.read_csv(StringIO(data))#, index_col=0)
'''

# Set up data directory
DataDir = "C:/Users/Admin/Documents/data/"
filename = "ThaiKAVideoProcess.xlsx"

# Set up output directory
OutputDir = "C:/Users/Admin/Documents/GitHub/KhanAcademyThailand/"

#Read data
df = pd.pandas.read_excel(DataDir+filename,sheetname='AllAvailableThaiVideos')

In [3]:
df[:10]

,Domain,DomainThai,Subject,SubjectThai,Topic,TopicThai,Tutorial,TutorialThai,ThaiCurriculumMapping,IsComplete,...,KASiteNameThai,KA site link,dub (Y/N)?,sub (Y/N)?,Original YoutubeID,Subbed Youtube Link,Dubbed Youtube Link,Dubbed Youtube ID,[YoutubeEnglishName] [[YoutubeThaiName]],Note
0,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,ประถมต้น,1.0,...,การนับจำนวนน้อยๆ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,y2-uaPiyoxc,https://www.youtube.com/watch?v=y2-uaPiyoxc,https://www.youtube.com/watch?v=UWrVntg0mvE,UWrVntg0mvE,NaN,NaN
1,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,ประถมต้น,1.0,...,การนับตามลำดับ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,PEeUTQ0Gri8,https://www.youtube.com/watch?v=PEeUTQ0Gri8,https://www.youtube.com/watch?v=4gVWDnuT-sU,4gVWDnuT-sU,NaN,NaN
2,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,ประถมต้น,1.0,...,ตารางจำนวน,https://www.khanacademy.org/math/early-math/cc...,Y,Y,9XZypM2Z3Ro,https://www.youtube.com/watch?v=9XZypM2Z3Ro,https://www.youtube.com/watch?v=XfldTHtzifo,XfldTHtzifo,NaN,NaN
3,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,ประถมต้น,1.0,...,จำนวนที่หายไประหว่าง 0 ถึง 120...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,1AqkBdCBm9o,https://www.youtube.com/watch?v=1AqkBdCBm9o,https://www.youtube.com/watch?v=F0a5lDts8mM,F0a5lDts8mM,NaN,NaN
4,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับในภาพ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,leDYnoNSvD4,https://www.youtube.com/watch?v=leDYnoNSvD4,https://www.youtube.com/watch?v=imcoyigzARU,imcoyigzARU,NaN,NaN
5,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับวาฬ แกะ และดอกไม้...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,I9S5CvSqb5A,https://www.youtube.com/watch?v=I9S5CvSqb5A,https://www.youtube.com/watch?v=lseJFKHN3ek,lseJFKHN3ek,NaN,NaN
6,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,ประถมต้น,1.0,...,การนับหมา หนู และคุกกี้...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,EUqhLxFccbM,https://www.youtube.com/watch?v=EUqhLxFccbM,https://www.youtube.com/watch?v=QJcLgMTEE4k,QJcLgMTEE4k,NaN,NaN
7,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,การเปรียบเทียบจำนวนสิ่งของ...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,__nkbr6DeTg,https://www.youtube.com/watch?v=__nkbr6DeTg,https://www.youtube.com/watch?v=jjhiWOpVEAE,jjhiWOpVEAE,NaN,NaN
8,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,การเปรียบเทียบจำนวนน้อยๆ บนเส้นจำนวน...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,tJrSILRXOUc,https://www.youtube.com/watch?v=tJrSILRXOUc,https://www.youtube.com/watch?v=1_K2prG61-Y,1_K2prG61-Y,NaN,NaN
9,Math by subject,คณิตศาสตร์ตามวิชา...,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,ประถมต้น,1.0,...,นับตามประเภท,https://www.khanacademy.org/math/early-math/cc...,Y,Y,UA975j_qsTQ,https://www.youtube.com/watch?v=UA975j_qsTQ,https://www.youtube.com/watch?v=beoLEZqtVGI,beoLEZqtVGI,NaN,NaN


We will use an interactive visualization tool called Bokeh. See an [overview](http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/quickstart/quickstart.ipynb) and [how to install](http://bokeh.pydata.org/en/0.11.0/docs/installation.html).

In [4]:
from bokeh.models import ColumnDataSource, HoverTool, OpenURL, TapTool, Div
from bokeh.plotting import figure, show
from bokeh.sampledata.periodic_table import elements
from bokeh.io import vplot

from bokeh.layouts import row, widgetbox, layout
from bokeh.models import Select, TextInput
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg

from bokeh.embed import components

In [5]:
#make output appeared in notebook
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

## Stage 2.1: Query some data

In [6]:
DomainList = ["คณิตศาสตร์ตามวิชา","คณิตศาสตร์ตามระดับชั้น","วิทยาศาสตร์","คอมพิวเตอร์","ศิลปะและมนุษยศาสตร์","เศรษฐศาสตร์และการเงิน","เตรียมสอบ","หน่วยงานที่ร่วมมือ","(ไม่มีโดเมน)"]

In [7]:
pd.DataFrame(df['SubjectThai'][df['DomainThai']==DomainList[7]].unique())

#pd.DataFrame(df['TutorialThai'][(df['DomainThai']=="คณิตศาสตร์ตามวิชา")&(df['SubjectThai']=="คณิตปฐมวัย")\
#                            &(df['TopicThai']=="การนับ")].unique())

,0
0,โรงเรียนแพทย์สแตนฟอร์ด...
1,23andMe
2,Pixar in a Box


In [8]:
pd.DataFrame(df['Domain'].unique())

,0
0,Math by subject
1,Math by grade
2,Science
3,Computing
4,Art & Humanities
5,Economics and finance
6,Test prep
7,Partner
8,(No domain)


## Stage 2.2: Child page prototype

In [9]:
header = """
<!DOCTYPE html>
<html lang="en">
  <head>
   <!-- Latest compiled and minified CSS -->
	<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">

	<!-- jQuery library -->
	<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js"></script>

	<!-- Latest compiled JavaScript -->
	<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>

	<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.2.min.css" type="text/css" />
        
		<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.2.min.js"></script>
		<script type="text/javascript">
			Bokeh.set_log_level("info");
		</script>
		<script type="text/javascript" src="C:/Users/Admin/Desktop/Insight/basicslider.js"></script>
		
	<style>
		
	</style>
	<meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
    <meta name="description" content="">
    <meta name="author" content="">
    <link rel="icon" href="../../favicon.ico">

    <title>Khan Academy Thailand</title>

    <!-- Bootstrap core CSS -->
    <link href="../../dist/css/bootstrap.min.css" rel="stylesheet">

    <!-- IE10 viewport hack for Surface/desktop Windows 8 bug -->
    <link href="../../assets/css/ie10-viewport-bug-workaround.css" rel="stylesheet">

    <!-- Just for debugging purposes. Don't actually copy these 2 lines! -->
    <!--[if lt IE 9]><script src="../../assets/js/ie8-responsive-file-warning.js"></script><![endif]-->
    <script src="../../assets/js/ie-emulation-modes-warning.js"></script>

    <!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
    <!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/html5shiv/3.7.3/html5shiv.min.js"></script>
      <script src="https://oss.maxcdn.com/respond/1.4.2/respond.min.js"></script>
    <![endif]-->

    <!-- Custom styles for this template -->
    <link href="carousel.css" rel="stylesheet">
    
    <!-- Google Analytics Tracker -->
	<script>
  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');

  ga('create', 'UA-97408983-1', 'auto');
  ga('send', 'pageview');

</script>
	
  </head>
<!-- NAVBAR
================================================== -->
  <body>
    <div class="navbar-wrapper">
      <div class="container">
		
		<nav class="navbar navbar-default">
  <div class="container-fluid">
    <!-- Brand and toggle get grouped for better mobile display -->
    <div class="navbar-header">
      <button type="button" class="navbar-toggle collapsed" data-toggle="collapse" data-target="#bs-example-navbar-collapse-1" aria-expanded="false">
        <span class="sr-only">Toggle navigation</span>
        <span class="icon-bar"></span>
        <span class="icon-bar"></span>
        <span class="icon-bar"></span>
      </button>
      <!--a class="navbar-brand" href="#"><img alt="Brand" src="KhanAcademyLogo.jpg" height="30"></a-->
	  <a class="navbar-brand" href="https://tponsukcharoen.github.io/KhanAcademyThailand/index.html#">Khan Academy Thailand</a>
    </div>

    <!-- Collect the nav links, forms, and other content for toggling -->
    <div class="collapse navbar-collapse" id="bs-example-navbar-collapse-1">
      <ul class="nav navbar-nav">
        <li class="dropdown">
          <a href="#" class="dropdown-toggle" data-toggle="dropdown" role="button" aria-haspopup="true" aria-expanded="false">วิชา <span class="caret"></span></a>
          <ul class="dropdown-menu">
			<li><a href="index.html#Math by subject">คณิตศาสตร์ตามวิชา</a></li>
			<li><a href="index.html#Math by grade">คณิตศาสตร์ตามระดับชั้น</a></li>
			<li><a href="index.html#Science">วิทยาศาสตร์</a></li>
			<li><a href="index.html#Computing">คอมพิวเตอร์</a></li>
			<li><a href="index.html#Art & Humanities">ศิลปะและมนุษยศาสตร์</a></li>
			<li><a href="index.html#Economics and finance">เศรษฐศาสตร์และการเงิน</a></li>
			<li><a href="index.html#Test prep">เตรียมสอบ</a></li>
			<li><a href="index.html#Partner">หน่วยงานที่ร่วมมือ</a></li>
			<li><a href="index.html#(No domain)">(ไม่มีโดเมน)</a></li>
          </ul>
        </li>
      </ul>
      <!--form class="navbar-form navbar-left">
        <div class="form-group">
          <input type="text" class="form-control" placeholder="หัวข้อ...">
        </div>
        <button type="submit" class="btn btn-default">ค้นหา</button>
      </form-->
	  <ul class="nav navbar-nav navbar-right">
        <li><a href="index.html#AboutUs">เกี่ยวกับเรา</a></li>
      </ul>
	  <!--li class="active"><a href="#">Link <span class="sr-only">(current)</span></a></li-->
      <!--li><a href="#">Link</a></li-->
      
    </div><!-- /.navbar-collapse -->
  </div><!-- /.container-fluid -->
</nav>

      </div>
    </div>

    <!-- Marketing messaging and featurettes
     ================================================== -->
    <!-- Wrap the rest of the page in another container to center all the content. -->
"""

In [10]:
footer = """
      </div>
      <!-- FOOTER -->
      <footer>
        <p class="pull-right"><a href="#">Back to top</a></p>
        <p>&copy; 2017 Khan Academy Thailand </p>
      </footer>
	  

    </div><!-- /.container -->


    <!-- Bootstrap core JavaScript
    ================================================== -->
    <!-- Placed at the end of the document so the pages load faster -->
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
    <script>window.jQuery || document.write('<script src="../../assets/js/vendor/jquery.min.js"><\/script>')</script>
    <script src="../../dist/js/bootstrap.min.js"></script>
    <!-- Just to make our placeholder images work. Don't actually copy the next line! -->
    <script src="../../assets/js/vendor/holder.min.js"></script>
    <!-- IE10 viewport hack for Surface/desktop Windows 8 bug -->
    <script src="../../assets/js/ie10-viewport-bug-workaround.js"></script>
  </body>
</html>
"""

In [11]:
for m in range(0,len(DomainList)):
    DomainName = DomainList[m]
    SubjectList = pd.DataFrame(df['SubjectThai'][df['DomainThai']==DomainName].unique())
    for l in range(0,len(SubjectList)):
        SubjectName = SubjectList[0][l]
        title = """
            <div class="container marketing">

                <!-- Three columns of text below the carousel -->
              <div class="row">
                <div class="col-lg-11">
                  <h2>""" + DomainName + " > " + SubjectName + \
        """</h2>
                </div><!-- /.col-lg-4 -->
              </div><!-- /.row -->
              
              <div id="accordion" role="tablist" aria-multiselectable="true">
    
              """
        tables = ""

        TopicList = pd.DataFrame(df['TopicThai'][(df['DomainThai']==DomainName)&(df['SubjectThai']==SubjectName)].unique())
        for k in range(0, len(TopicList)):
            TopicName = TopicList[0][k]
            table_header = """
            <div class="card">
    <div class="card-header" role="tab" id="heading_""" + str(m) + "_" + str(l) + "_" + str(k) +"""">
      <h5 class="mb-0">
        <a data-toggle="collapse" data-parent="#accordion" href="#collapse_""" \
            + str(m) + "_" + str(l) + "_" + str(k)+ """" aria-expanded="true" aria-controls="collapse_""" \
            + str(m) + "_" + str(l) + "_" + str(k)+ """"><h3>""" + TopicName + """</h3>
        </a>
      </h5>
    </div>
    <div id="collapse_""" + str(m) + "_" + str(l) + "_" + str(k) + """" class="collapse" role="tabpanel" aria-labelledby="heading_"""+ str(m) + "_" + str(l) + "_" + str(k) +"""">
      <div class="card-block">
                <div class="panel panel-info">
                      <!-- Default panel contents -->
                      <div class="panel-heading">""" + TopicName + \
                """</div>
                      <!-- Table -->
                      <table class="table">
                      """

            table_footer = """
                      </table>
                    </div>
                </div>
                </div>
                </div>
                """
            TutorialList = pd.DataFrame(df['TutorialThai'][(df['DomainThai']==DomainName)&(df['SubjectThai']==SubjectName)&\
                                                       (df['TopicThai']==TopicName)].unique())
            table_body = "" 
            for j in range(0,len(TutorialList)):
                TutorialName = TutorialList[0][j]
                #Add tutorial names in the table
                #table_body += """<tr><td><b>""" + TutorialName +"""</b></td></tr>"""
                KASiteNameThaiList = pd.DataFrame(df['KASiteNameThai'][(df['DomainThai']==DomainName)&(df['SubjectThai']==SubjectName)&\
                                                           (df['TopicThai']==TopicName)&(df['TutorialThai']==TutorialName)].unique())
                for i in range(0,len(KASiteNameThaiList)):
                    if i == 0:
                        table_body += """<tr><td><b>""" + TutorialName +"""</b></td>"""
                    else:
                        table_body += """<tr><td></td>"""
                    KASiteNameThai = KASiteNameThaiList[0][i]
                    table_body += """<td>""" + KASiteNameThai +"""</td><td>| """ 
                    df_small = df[(df['DomainThai']==DomainName)&(df['SubjectThai']==SubjectName)&(df['TopicThai']==TopicName)\
                            &(df['TutorialThai']==TutorialName)&(df['KASiteNameThai']==KASiteNameThai)].reset_index()
                    if (df_small['sub (Y/N)?'][0] == 'Y'):
                        table_body += """<a href="https://www.youtube.com/watch?v=""" + df_small['Original YoutubeID'][0] +"""">คลิปพร้อมซับไตเติ้ล</a> | """
                    if (df_small['dub (Y/N)?'][0] == 'Y'):
                        table_body += """<a href="https://www.youtube.com/watch?v=""" + df_small['Dubbed Youtube ID'][0] +"""">คลิปพากย์ภาษาไทย</a> | """
                    if (df_small['IsComplete'][0]==-1):
                        table_body += """ยังไม่เปิดให้แปลซับไตเติ้ลภาษาไทย | """
                    else:
                        if not (df_small['sub (Y/N)?'][0] == 'Y')|(df_small['dub (Y/N)?'][0] == 'Y'):
                            table_body += """ยังไม่มีซับไตเติ้ลภาษาไทย ช่วยแปลได้ <a href="https://www.youtube.com/timedtext_video?v=""" +df_small['Original YoutubeID'][0] + """">ที่นี่</a> | """
                    table_body += """</td></tr>"""

            table = table_header + table_body + table_footer
            tables += table

        r_all = header + title + tables + footer

        text_file = open(OutputDir+ str(m)+ "_"+str(l)+".html", "w")
        text_file.write("%s" % r_all)
        text_file.close()


In [12]:
df_small

,index,Domain,DomainThai,Subject,SubjectThai,Topic,TopicThai,Tutorial,TutorialThai,ThaiCurriculumMapping,...,KASiteNameThai,KA site link,dub (Y/N)?,sub (Y/N)?,Original YoutubeID,Subbed Youtube Link,Dubbed Youtube Link,Dubbed Youtube ID,[YoutubeEnglishName] [[YoutubeThaiName]],Note
0,5476,(No domain),(ไม่มีโดเมน),Resources,แหล่งข้อมูล,Classroom case studies,กรณีศึกษาในห้องเรียน...,Primary level (grades 1 - 8),ชั้นประถม (เกรด 1-8),ทั่วไป,...,คานอะคาเดมี่ในโรงเรียนเขตลอส อัลตอส...,https://www.khanacademy.org/resources/classroo...,NaN,NaN,eJQzBJ6DtoY,NaN,NaN,NaN,NaN,NaN


## Stage 3: What's next?

Now the output files are child HTML pages. Note that the homepage needs to be upated manually. At this point, Google Analytics tracking code is added.

There are some ideas to develop this further:

1. Integrate seach capability
2. Add more content from other website. Tool to scrap URL: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ 
4. Do pop-up servey questions

*Note to myself:* I haven't tracked Pomodoros. I get a flow from this work. Let's try new things next time.